In [ ]:
from  AISecClient import AISecClient
import os
from transformers import AutoImageProcessor, ResNetForImageClassification
from time import sleep
from datasets import load_dataset
import torch
from datetime import datetime
from art.estimators.classification import BlackBoxClassifier
from art.attacks.evasion import HopSkipJump
from art.utils import to_categorical
import numpy as np
from matplotlib import pyplot as plt

def process_array(input_array, low, high):
    # Ensure the input is a numpy array
    input_array = np.asarray(input_array)

    # Generate a random matrix of the same shape
    random_matrix = np.random.uniform(low=low, high=high, size=input_array.shape)

    # Multiply the input array by the random matrix
    multiplied_array = input_array * random_matrix

    # Normalize the multiplied array back to the range 0.0 to 255.0
    min_val = multiplied_array.min()
    max_val = multiplied_array.max()
    normalized_array = 255 * (multiplied_array - min_val) / (max_val - min_val)
    normalized_array = normalized_array.astype(np.uint8)
    return normalized_array


# setup session to use for demo
# ADD CLIENT ID AND CLIENT SECRET TO RUN NOTEBOOK
client_id = os.getenv('HIDDENLAYER_CLIENT_ID')
client_secret = os.getenv('HIDDENLAYER_CLIENT_SECRET')
client = AISecClient(client_id, client_secret)
 
# load dataset
dataset = load_dataset("zh-plus/tiny-imagenet")
# load the microsoft/resnet-50 model
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
sensor = client.get_sensor_by_name_version("microsoft/resnet-50", 1)
 
test_image = dataset["valid"]["image"][3]
display(test_image)
 
image_array = np.array(test_image)
pimg = processor(image_array, return_tensors="pt")
low = 0.9
high = 1.1
with torch.no_grad():
    for i in range(20):
        image_array = process_array(image_array, low, high) 
        t = processor(image_array, return_tensors="pt")['pixel_values']
        p = model(t.float())
        prob = p.logits
        l = prob.argmax(-1).item()
        tl = model.config.id2label[l]
        print (i, l,tl)
        client.submit_to_mldr(sensor["sensor_id"], "resnet-50-demo", pimg["pixel_values"].numpy(),
                              p.logits.numpy(), predictions=[l])
 
        low *= 0.995
        high *= 1.005
        plt.imshow(image_array)
        plt.show()
